In [1]:
import matplotlib.pyplot as plt

import traceback
import numpy as np
import tushare as ts
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D  #绘制三D图形

from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction import FeatureHasher, DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from lutils.stock import LTdxHq

In [3]:
# 获取股票基本信息，包括 PE、PB 值
ts.set_token('42731ca565c5d019007ef5cd7db7808757b2ea3fdbfb31d4f7b61444b')
pro = ts.pro_api()

df_base = pro.stock_basic() # ts.get_stock_basics()
# df_base['code'] = df_base.index

# 获取股票当天数据，包括当前股价
df_todays = ts.get_today_all()
# df_todays['code'] = df_todays.index

df_roe = ts.get_report_data(2022, 3)
# df_roe['code'] = df_roe.index

[Getting data:]####################################################本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
[Getting data:]##################################################################################

In [4]:
df_base['code'] = df_base.index
df_todays['code'] = df_todays.index
df_roe['code'] = df_roe.index

In [5]:
# 整合股价与 PE、PB 数据
df = pd.merge(df_todays, df_base, how='left', on=['code'])
df = pd.merge(df, df_roe, how='left', on=['code'])

df.dropna()
result = df[(0 < df['pb']) 
            & (df['pb'] < 8)
            & (-20 < df['roe']) 
            & (df['roe'] < 30) 
            & (0 < df['trade']) 
            & (df['trade'] < 70)
            & (-100<df['per']) 
            & (df['per']<200)]

# result = df[(-100 < df['per']) & (df['per']<200) 
# #             &(0<df['pb_x'])&(df['pb_x']<10)
#             &(0<df['pb'])&(df['pb']<10)
# #              &(roe_describe['25%']<df['roe'])&(df['roe']<roe_describe['75%'])
#              &(0<df['trade'])&(df['trade']<100)]

# X = result[['pb', 'roe', 'trade', 'code', 'industry', 'market']]

In [6]:
df.columns

Index(['code', 'name_x', 'changepercent', 'trade', 'open', 'high', 'low',
       'settlement', 'volume', 'turnoverratio', 'amount', 'per', 'pb',
       'mktcap', 'nmc', 'ts_code', 'symbol', 'name_y', 'area', 'industry',
       'market', 'list_date', 'name', 'eps', 'eps_yoy', 'bvps', 'roe', 'epcf',
       'net_profits', 'profits_yoy', 'distrib', 'report_date'],
      dtype='object')

In [7]:
result.ts_code.shape

(3890,)

In [8]:
result.ts_code.unique().shape

(3890,)

In [ ]:
# ddf = pd.read_pickle('d:/ddf_.pkl').dropna()

In [9]:
result.ts_code

0       000001.SZ
2       000004.SZ
3       000005.SZ
4       000006.SZ
5       000007.SZ
          ...    
4856    688767.SH
4857    688768.SH
4858    688772.SH
4859    688776.SH
4860    688777.SH
Name: ts_code, Length: 3890, dtype: object

In [10]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in result.ts_code.values: # ddf.tic.unique(): #
    try:
        code = code[:-3]
        df = ltdxhq.get_k_data_daily(code, start='2016-01-01') # 2012-01-01

        if indexs is None:
            indexs = df.index
        else:
            indexs = indexs.union(df.index)

    #     df = df.assign(date = df.index)
    #     df = df.assign(day = df.index.weekday)
    #     df.date = df.date.dt.strftime('%Y-%m-%d')
        df = df.assign(tic = code)
    #     df.index = range(df.shape[0])
        
        if df.shape[0] > 90:
            dfs.append(df)
            print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))
        else:
            print('---pass---- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))
    except Exception as ex:
#         print(traceback.format_exc())
        print('error code: %s' % code)

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
ddf = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 000001 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000004 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000005 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000006 min: 2018-03-08 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000007 min: 2018-03-09 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000008 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000009 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000010 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000011 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000012 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000014 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000016 min: 2018-01-02 00:00:00 max: 

----------- over 000513 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000514 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000516 min: 2018-04-17 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000517 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000518 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000519 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000520 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000521 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000523 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000524 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000526 min: 2018-01-10 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000531 min: 2018-01-02 00:00:00 max: 

----------- over 000712 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000713 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000715 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000719 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000720 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000722 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000727 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000729 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000732 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000733 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000735 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000736 min: 2018-01-02 00:00:00 max: 

----------- over 000962 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000963 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000966 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000968 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000969 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000972 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000973 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000978 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000981 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000985 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000987 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 000988 min: 2018-01-02 00:00:00 max: 

----------- over 002068 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002072 min: 2018-05-21 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002073 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002075 min: 2018-11-16 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002076 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002077 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002082 min: 2018-07-19 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002083 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002086 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002087 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002088 min: 2018-01-02 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 002089 min: 2018-01-02 00:00:00 max: 

...

----------- over 605333 min: 2020-08-18 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605336 min: 2020-10-19 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605337 min: 2021-02-08 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605339 min: 2021-05-18 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605358 min: 2020-09-11 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605365 min: 2021-07-20 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605366 min: 2020-08-12 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605369 min: 2020-09-16 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605376 min: 2020-12-08 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605378 min: 2021-04-12 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605389 min: 2021-03-22 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 605398 min: 2021-01-21 00:00:00 max: 

----------- over 688096 min: 2020-04-09 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688097 min: 2021-05-12 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688099 min: 2019-08-08 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688100 min: 2020-01-21 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688101 min: 2019-11-15 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688102 min: 2022-03-16 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688103 min: 2021-09-10 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688105 min: 2021-11-15 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688106 min: 2020-06-16 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688107 min: 2021-11-12 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688108 min: 2019-10-30 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688109 min: 2021-03-30 00:00:00 max: 

----------- over 688230 min: 2021-12-01 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688232 min: 2021-11-17 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688233 min: 2020-02-21 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688234 min: 2022-01-12 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688235 min: 2021-12-15 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688236 min: 2021-12-30 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688237 min: 2022-07-01 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688238 min: 2022-03-22 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688239 min: 2021-07-05 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688244 min: 2022-10-19 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688246 min: 2021-12-14 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688247 min: 2022-08-25 00:00:00 max: 

----------- over 688383 min: 2021-04-28 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688385 min: 2021-08-04 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688387 min: 2022-09-26 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688388 min: 2019-07-22 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688389 min: 2019-11-05 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688390 min: 2020-09-04 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688391 min: 2022-09-13 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688392 min: 2022-09-27 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688393 min: 2020-08-20 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688396 min: 2020-02-27 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688398 min: 2020-02-11 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688399 min: 2019-12-05 00:00:00 max: 

----------- over 688676 min: 2021-03-09 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688677 min: 2021-02-26 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688679 min: 2020-12-25 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688681 min: 2021-06-16 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688682 min: 2021-04-20 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688683 min: 2021-04-12 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688685 min: 2021-05-13 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688686 min: 2020-12-31 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688687 min: 2021-02-08 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688689 min: 2021-01-27 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688690 min: 2021-06-23 00:00:00 max: 2022-11-11 00:00:00 -----------
----------- over 688696 min: 2021-03-03 00:00:00 max: 

In [11]:
ddf.to_pickle('d:/ddf.pkl')

In [11]:
ddf.shape

(4581940, 9)

In [12]:
ddf

,open,close,high,low,volume,amount,tic,date,day
0,12.46,12.79,13.00,12.43,208159200.0,2.856543e+09,000001,2018-01-02,1
1,12.82,12.44,12.94,12.32,296249792.0,4.006220e+09,000001,2018-01-03,2
2,12.43,12.37,12.48,12.26,185450896.0,2.454543e+09,000001,2018-01-04,3
3,12.33,12.41,12.46,12.27,121031200.0,1.603289e+09,000001,2018-01-05,4
4,12.37,12.10,12.41,12.00,215862000.0,2.806099e+09,000001,2018-01-08,0
...,...,...,...,...,...,...,...,...,...
1175,92.65,89.40,93.80,89.18,4868469.0,4.400796e+08,688777,2022-11-07,0
1176,89.47,92.25,92.80,89.46,5228649.0,4.763130e+08,688777,2022-11-08,1
1177,91.31,92.62,94.11,90.24,3337525.0,3.092194e+08,688777,2022-11-09,2
1178,91.41,90.89,93.17,90.32,2674008.0,2.442208e+08,688777,2022-11-10,3


In [13]:
ddf.dropna().to_pickle('d:/ddf.pkl')

In [5]:
dddf = pd.read_pickle('d:/ddf.pkl').dropna()

In [8]:
len(dddf.tic.unique().tolist())

3547

In [9]:
dddf.tic.unique()

array(['000002', '000004', '000005', ..., '688786', '688787', '688789'],
      dtype=object)

In [13]:
df1124 = pd.read_pickle('d:/ddf_1124.pkl').dropna()

In [16]:
dddf[dddf.tic == '000002']

,open,close,high,low,volume,amount,tic,date,day
121,17.23,17.23,17.23,17.23,4.266300e+06,9.381500e+07,000002,2016-07-04,0
122,15.50,15.50,15.50,15.50,1.990576e+08,3.939350e+09,000002,2016-07-05,1
123,14.96,15.51,16.28,14.96,1.028373e+09,2.010649e+10,000002,2016-07-06,2
124,14.96,14.74,15.06,14.66,4.300225e+08,8.133977e+09,000002,2016-07-07,3
125,14.57,14.69,14.87,14.27,3.124436e+08,5.799513e+09,000002,2016-07-08,4
...,...,...,...,...,...,...,...,...,...
1490,20.30,20.70,20.91,20.10,9.333739e+07,1.912526e+09,000002,2022-02-21,0
1491,20.51,20.64,20.94,20.40,8.677726e+07,1.788757e+09,000002,2022-02-22,1
1492,20.65,20.26,20.69,20.02,9.169187e+07,1.852695e+09,000002,2022-02-23,2
1493,20.15,19.84,20.38,19.60,8.797944e+07,1.752828e+09,000002,2022-02-24,3


In [15]:
df1124[df1124.tic == '000002']

,open,close,high,low,volume,amount,tic,date,day
121,16.22,16.22,16.22,16.22,4.266300e+06,9.381500e+07,000002,2016-07-04,0
122,14.59,14.59,14.59,14.59,1.990576e+08,3.939350e+09,000002,2016-07-05,1
123,14.08,14.60,15.32,14.08,1.028373e+09,2.010649e+10,000002,2016-07-06,2
124,14.08,13.88,14.17,13.80,4.300225e+08,8.133977e+09,000002,2016-07-07,3
125,13.72,13.83,14.00,13.43,3.124436e+08,5.799513e+09,000002,2016-07-08,4
...,...,...,...,...,...,...,...,...,...
1671,15.87,15.81,16.00,15.57,8.622672e+07,1.359628e+09,000002,2022-11-17,3
1672,15.78,15.65,15.93,15.58,6.751475e+07,1.061067e+09,000002,2022-11-18,4
1673,15.57,15.57,15.60,15.08,7.632241e+07,1.170549e+09,000002,2022-11-21,0
1674,15.80,15.60,15.95,15.39,8.283687e+07,1.295600e+09,000002,2022-11-22,1
